In [7]:
function sis_triang_sup(A::Matrix{Float64}, b::Vector{Float64})
    X = zeros(length(b))
    for i in length(b):-1:1
        x = (b[i] - sum(X .* A[i, :])) / A[i, i]
        X[i] = x
    end
    return round.(X, digits=5)
end

sis_triang_sup (generic function with 1 method)

In [185]:
A = [5. -2. 6. 1.; 0. 3. 7. -4.; 0. 0. 4. 5.; 0. 0. 0. 2.]

4×4 Matrix{Float64}:
 5.0  -2.0  6.0   1.0
 0.0   3.0  7.0  -4.0
 0.0   0.0  4.0   5.0
 0.0   0.0  0.0   2.0

In [36]:
convert.(Float32, A)

4×4 Matrix{Float32}:
 5.0  -2.0  6.0   1.0
 0.0   3.0  7.0  -4.0
 0.0   0.0  4.0   5.0
 0.0   0.0  0.0   2.0

In [183]:
b = [1.; -2.; 28.; 8.]

4-element Vector{Float64}:
  1.0
 -2.0
 28.0
  8.0

In [186]:
sis_triang_sup(A, b)

4-element Vector{Float64}:
 -3.0
  0.0
  2.0
  4.0

In [149]:
function eliminacao_gauss(A::Matrix{Float64}, b::Vector{Float64})

    M = hcat(A, b)
    
    function pivotamento(c::Int64, l::Int64, M::Matrix{Float64})
        # c para coluna atual e l para linha atual
        vetor = abs.(M[l:end, c])
        max = argmax(vetor)
        max_index = max + l - 1
        M[max_index, :], M[l, :] = M[l, :], M[max_index, :]    
    end
    
    for i in 1:length(M[:, 1])
        pivotamento(i, i, M)
        pivot_line = M[i, :]
        for j in (i+1):(length(M[1, :])-1)
            m = M[j, i]/pivot_line[i]
            M[j, :] -= m * pivot_line
        end
    end

    return M[:, 1:(end-1)], M[:, end]
    
end

eliminacao_gauss (generic function with 1 method)

In [43]:
M = [2. 3. -1.; 4. 4. -3.; 2. -3. 1.]

3×3 Matrix{Float64}:
 2.0   3.0  -1.0
 4.0   4.0  -3.0
 2.0  -3.0   1.0

In [44]:
N = [5.; 3.; -1.]

3-element Vector{Float64}:
  5.0
  3.0
 -1.0

In [22]:
V = eliminacao_gauss(M, N)

([4.0 4.0 -3.0; 0.0 -5.0 2.5; 0.0 0.0 1.0], [3.0, -2.5, 3.0])

In [23]:
V[1]

3×3 Matrix{Float64}:
 4.0   4.0  -3.0
 0.0  -5.0   2.5
 0.0   0.0   1.0

In [24]:
V[2]

3-element Vector{Float64}:
  3.0
 -2.5
  3.0

In [25]:
X = sis_triang_sup(V[1], V[2])

3-element Vector{Float64}:
 1.0
 2.0
 3.0

In [26]:
inv(M) * N

3-element Vector{Float64}:
 0.9999999999999998
 2.0
 3.0

In [38]:
A1 = [1. 2. 3.; -3. 2. 1.; 1. -1. 1.]
b1 = [6.; 0.; 1.];

In [30]:
R = eliminacao_gauss(A1, b1)

([-3.0 2.0 1.0; 0.0 2.6666666666666665 3.3333333333333335; 0.0 0.0 1.75], [0.0, 6.0, 1.7500000000000002])

In [31]:
R[1]

3×3 Matrix{Float64}:
 -3.0  2.0      1.0
  0.0  2.66667  3.33333
  0.0  0.0      1.75

In [34]:
sis_triang_sup(R[1], R[2])

3-element Vector{Float64}:
 1.0
 1.0
 1.0

In [150]:
function resolver_sistema(A::Matrix{Float64}, b::Vector{Float64})
    I = eliminacao_gauss(A, b)
    return sis_triang_sup(I[1], I[2])
end

resolver_sistema (generic function with 1 method)

In [45]:
resolver_sistema(M, N)

3-element Vector{Float64}:
 1.0
 2.0
 3.0

In [138]:
A = rand(1:100, (6,6))
A = convert.(Float64, A)

6×6 Matrix{Float64}:
 96.0  39.0  28.0  26.0  90.0  15.0
 97.0  23.0  34.0  99.0  40.0  69.0
 83.0  33.0  29.0  76.0  86.0  76.0
 71.0  34.0  79.0  29.0  27.0  81.0
 14.0  97.0  69.0  59.0  73.0  78.0
 99.0  16.0   5.0  30.0  43.0  30.0

In [139]:
b = rand(1:100, 6)
b = convert.(Float64, b)

6-element Vector{Float64}:
 22.0
 21.0
 44.0
 37.0
 64.0
 53.0

In [52]:
using BenchmarkTools

In [56]:
function teste(n::Int64, range) 
    A = rand(range, (n,n))
    A = convert.(Float64, A)
    b = rand(range, n)
    b = convert.(Float64, b)
    return resolver_sistema(A, b)
end

teste (generic function with 1 method)

In [60]:
t1 = @benchmark teste(8, 10:30)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  13.000 μs …  1.498 ms  ┊ GC (min … max): 0.00% … 97.33%
 Time  (median):     14.000 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   17.482 μs ± 29.091 μs  ┊ GC (mean ± σ):  3.21% ±  1.95%

  ▄▇█▆▅▁▁▁▁                      ▁▂▂▂▁▂▂▂▁▂▁▂▁▁▁▁ ▁           ▂
  █████████▇▆▆▅▄▅▅▅▃▄▄▅▄▄▄▄▄▅▅▆▇███████████████████████▇▇▆▇▇▄ █
  13 μs        Histogram: log(frequency) by time      33.6 μs <

 Memory estimate: 21.09 KiB, allocs estimate: 158.

In [61]:
t2 = @benchmark teste(16, 10:30)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  44.300 μs …  1.307 ms  ┊ GC (min … max): 0.00% … 94.32%
 Time  (median):     46.300 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   52.200 μs ± 52.964 μs  ┊ GC (mean ± σ):  4.51% ±  4.39%

  ▅█▇▅▃▁▁▁                                             ▁▁▁▁   ▂
  █████████▇▇▇▆▇▆▅▅▅▅▅▅▄▅▅▅▆▅▄▄▃▄▃▁▁▃▄▅█▆▆▆▄▃▁▃▁▁▃▄▅▅▇██████▇ █
  44.3 μs      Histogram: log(frequency) by time       109 μs <

 Memory estimate: 99.25 KiB, allocs estimate: 498.

In [64]:
ratio(mean(t2), mean(t1))

BenchmarkTools.TrialRatio: 
  time:             2.9860059296326615
  gctime:           4.195522254928196
  memory:           4.705185185185186
  allocs:           3.151898734177215

In [65]:
t3 = @benchmark teste(32, 10:30)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  188.200 μs …  1.319 ms  ┊ GC (min … max): 0.00% … 62.75%
 Time  (median):     193.700 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   208.273 μs ± 89.915 μs  ┊ GC (mean ± σ):  4.56% ±  8.72%

  █▄▃▁                                                         ▁
  ████▇▅▅▅▅▅▄▃▃▅▆▅▃▁▁▁▁▁▁▁▄▇▆▆▆▄▃▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▄▅▇ █
  188 μs        Histogram: log(frequency) by time       902 μs <

 Memory estimate: 598.00 KiB, allocs estimate: 1754.

In [66]:
ratio(mean(t3), mean(t1))

BenchmarkTools.TrialRatio: 
  time:             11.913796091796618
  gctime:           16.937311568994737
  memory:           28.34962962962963
  allocs:           11.10126582278481

In [81]:
2 ^ (1.789)

3.4557527496011087

In [80]:
4 ^ (1.789)

11.942227066375622

In [83]:
t4 = @benchmark teste(64, 10:30)

BenchmarkTools.Trial: 4683 samples with 1 evaluation.
 Range (min … max):  970.300 μs …   5.478 ms  ┊ GC (min … max): 0.00% …  0.00%
 Time  (median):     994.700 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.063 ms ± 236.193 μs  ┊ GC (mean ± σ):  4.83% ± 10.34%

  ▅█▇▅▄▂▂                                          ▁▁▁▁▁▁       ▁
  ████████▇▆▇▆▅▅▃▅▅▃▃▃▁▁▄▁▁▃▄▁▁▄▁▄▁▁▁▁▁▁▃▃▁▁▁▃▃▅▆▇████████▇▇▆▆▆ █
  970 μs        Histogram: log(frequency) by time       1.72 ms <

 Memory estimate: 3.70 MiB, allocs estimate: 6574.

In [84]:
ratio(mean(t4), mean(t1))

BenchmarkTools.TrialRatio: 
  time:             60.77976150843191
  gctime:           91.5350880275624
  memory:           179.71555555555557
  allocs:           41.607594936708864

In [89]:
8 ^ (1.975)

60.75789574083324

In [152]:
using LinearAlgebra

function decompLU(A::Matrix{Float64})
    U = copy(A)
    n = length(A[1,:])
    L = zeros(n, n)
    L[diagind(A)] .= 1   

    for i in 1:n
        #pivotamento(i, i, U)
        pivot_line = U[i, :]
        for j in (i+1):n
            m = U[j, i]/pivot_line[i]
            L[j, i] = m
            U[j, :] -= m * pivot_line
        end
    end 
    return L, U
end


decompLU (generic function with 1 method)

In [75]:
X = decompLU(A)

([1.0 0.0 … 0.0 0.0; 3.0 1.0 … 0.0 0.0; … ; 6.357142857142857 2.039442039442039 … 1.0 0.0; 4.5 1.4208754208754208 … 2.3499663424500894 1.0], [14.0 100.0 … 68.0 28.0; 0.0 -297.0 … -137.0 -48.0; … ; 0.0 0.0 … 259.3752590355107 -273.09053071546555; 0.0 0.0 … 0.0 157.76628202313913])

In [96]:
A

6×6 Matrix{Float64}:
 14.0  100.0  80.0  84.0  68.0  28.0
 42.0    3.0  44.0  79.0  67.0  36.0
 43.0   15.0  34.0  31.0   7.0  82.0
  2.0   79.0  61.0  79.0   9.0  39.0
 89.0   30.0  54.0  49.0  12.0  83.0
 63.0   28.0  16.0   4.0  55.0  41.0

In [77]:
X[1]

6×6 Matrix{Float64}:
 1.0        0.0        0.0       0.0      0.0      0.0
 3.0        1.0        0.0       0.0      0.0      0.0
 3.07143    0.983646   1.0       0.0      0.0      0.0
 0.142857  -0.217893  -0.362816  1.0      0.0      0.0
 6.35714    2.03944    2.89861   3.74737  1.0      0.0
 4.5        1.42088    3.46245   7.89552  2.34997  1.0

In [78]:
X[2]

6×6 Matrix{Float64}:
 14.0   100.0    80.0       84.0        68.0       28.0
  0.0  -297.0  -196.0     -173.0      -137.0      -48.0
  0.0     0.0   -18.9197   -56.8292    -67.0976    43.215
  0.0     0.0     0.0        8.68592   -54.9097    40.2202
  0.0     0.0     0.0        0.0       259.375   -273.091
  0.0     0.0     0.0        0.0         0.0      157.766

In [79]:
X[1] * X[2]

6×6 Matrix{Float64}:
 14.0  100.0  80.0  84.0  68.0  28.0
 42.0    3.0  44.0  79.0  67.0  36.0
 43.0   15.0  34.0  31.0   7.0  82.0
  2.0   79.0  61.0  79.0   9.0  39.0
 89.0   30.0  54.0  49.0  12.0  83.0
 63.0   28.0  16.0   4.0  55.0  41.0

In [83]:
function pivotamento(c::Int64, l::Int64, M::Matrix{Float64})
    # c para coluna atual e l para linha atual
    vetor = abs.(M[l:end, c])
    max = argmax(vetor)
    max_index = max + l - 1
    M[max_index, :], M[l, :] = M[l, :], M[max_index, :]    
end

pivotamento (generic function with 1 method)

In [103]:
#pivotamento(1, 1, A)

([89.0, 30.0, 54.0, 49.0, 12.0, 83.0], [89.0, 30.0, 54.0, 49.0, 12.0, 83.0])

In [109]:
A

6×6 Matrix{Float64}:
 78.0  24.0  82.0  61.0  44.0  25.0
 17.0  59.0  42.0  72.0   9.0  21.0
 79.0  91.0  48.0  46.0  42.0  52.0
 53.0  14.0  20.0  98.0  92.0  88.0
 56.0  88.0  97.0  10.0  89.0  46.0
  5.0  55.0  76.0  32.0  45.0  73.0

In [170]:
X = decompLU(A)

([1.0 0.0 … 0.0 0.0; 1.0104166666666667 1.0 … 0.0 0.0; … ; 0.14583333333333334 -5.565714285714286 … 1.0 0.0; 1.03125 1.4761904761904763 … -0.08491901486422106 1.0], [96.0 39.0 … 90.0 15.0; 0.0 -16.40625 … -50.9375 53.84375; … ; 0.0 0.0 … -258.6572639918164 -224.85209147944738; 0.0 0.0 … 0.0 60.56076135735984])

In [171]:
X[1]*X[2]

6×6 Matrix{Float64}:
 96.0  39.0  28.0  26.0  90.0  15.0
 97.0  23.0  34.0  99.0  40.0  69.0
 83.0  33.0  29.0  76.0  86.0  76.0
 71.0  34.0  79.0  29.0  27.0  81.0
 14.0  97.0  69.0  59.0  73.0  78.0
 99.0  16.0   5.0  30.0  43.0  30.0

In [172]:
A - X[1]*X[2]

6×6 Matrix{Float64}:
 0.0  0.0           0.0           0.0           0.0           0.0
 0.0  0.0           0.0           0.0           0.0           0.0
 0.0  0.0           0.0           0.0           0.0           0.0
 0.0  0.0           0.0           0.0          -2.84217e-14   0.0
 0.0  0.0          -1.42109e-14  -1.13687e-13   0.0           0.0
 0.0  1.77636e-15   5.32907e-15  -3.90799e-14   7.10543e-15  -3.19744e-14

In [173]:
X[1]

6×6 Matrix{Float64}:
 1.0        0.0         0.0      0.0        0.0       0.0
 1.01042    1.0         0.0      0.0        0.0       0.0
 0.864583   0.0438095   1.0      0.0        0.0       0.0
 0.739583  -0.314286   13.2301   1.0        0.0       0.0
 0.145833  -5.56571    21.2894   0.965721   1.0       0.0
 1.03125    1.47619    -7.1124  -0.400795  -0.084919  1.0

In [174]:
X[2]

6×6 Matrix{Float64}:
 96.0   39.0     28.0        26.0       90.0       15.0
  0.0  -16.4062   5.70833    72.7292   -50.9375    53.8438
  0.0    0.0      4.54159    50.3346    10.419     60.6724
  0.0    0.0      0.0      -633.304   -193.417   -715.874
  0.0    0.0      0.0         0.0     -258.657   -224.852
  0.0    0.0      0.0         0.0        0.0       60.5608

In [175]:
det(A)/det(X[2])

0.9999999999999993

In [20]:
function sis_triang_inf(A::Matrix{Float64}, B::Vector{Float64})
    X = zeros(length(B))
    X[1] = B[1]/A[1,1]
    for i in 2:length(B)
        soma = 0
        el = A[i, 1:(i-1)] .* X[1:(i-1)]
        soma += sum(el)
        X[i] = (B[i] - soma)/A[i,i]
    end
    return X
end


sis_triang_inf (generic function with 1 method)

In [176]:
V = sis_triang_inf(X[1], b)

6-element Vector{Float64}:
  1.0
 -0.01041666666666674
  0.1358730158730158
 -1.5404725290088077
 -0.6087964674793045
  0.2813977050069125

In [177]:
N = sis_triang_sup(X[2], V)

6-element Vector{Float64}:
  0.00865
  0.00994
 -0.00274
 -0.00231
 -0.00169
  0.00465

In [167]:
A

6×6 Matrix{Float64}:
 96.0  39.0  28.0  26.0  90.0  15.0
 97.0  23.0  34.0  99.0  40.0  69.0
 83.0  33.0  29.0  76.0  86.0  76.0
 71.0  34.0  79.0  29.0  27.0  81.0
 14.0  97.0  69.0  59.0  73.0  78.0
 99.0  16.0   5.0  30.0  43.0  30.0

In [168]:
b

6-element Vector{Float64}:
 22.0
 21.0
 44.0
 37.0
 64.0
 53.0

In [148]:
inv(A) * b

6-element Vector{Float64}:
  0.4053152106889447
  1.2321485499685036
 -1.2426093090294061
 -0.7257213041998618
 -0.319332709775346
  1.1625136511260397

In [169]:
resolver_sistema(A, b)

6-element Vector{Float64}:
  0.40532
  1.23215
 -1.24261
 -0.72572
 -0.31933
  1.16251

In [1]:
A = [16 5; 1 2]

2×2 Matrix{Int64}:
 16  5
  1  2

In [2]:
inv(A)

2×2 Matrix{Float64}:
  0.0740741  -0.185185
 -0.037037    0.592593

In [3]:
[1 0; 1/16 1] * [16 5; 0 4.875]

2×2 Matrix{Float64}:
 16.0  5.0
  1.0  5.1875

In [30]:
G =[1 0 0; 1/2 1 0; 0 4/5.5 1] * [2 1 0; 0 5.5 4; 0 0 8.09]

3×3 Matrix{Float64}:
 2.0  1.0   0.0
 1.0  6.0   4.0
 0.0  4.0  10.9991

In [31]:
inv(G)

3×3 Matrix{Float64}:
  0.5618     -0.123599    0.0449489
 -0.123599    0.247198   -0.0898977
  0.0449489  -0.0898977   0.123609

In [26]:
l = [1 0 0; 3/5 1 0; 1/5 -3 1]

3×3 Matrix{Float64}:
 1.0   0.0  0.0
 0.6   1.0  0.0
 0.2  -3.0  1.0

In [21]:
b = [0. ; 0. ; 1.]

3-element Vector{Float64}:
 0.0
 0.0
 1.0

In [27]:
sis_triang_sup(l, b)

3-element Vector{Float64}:
 0.0
 0.0
 1.0

In [23]:
sis_triang_inf([5 2 1; 0 -0.2 3.4; 0 0 13], b)

3-element Vector{Float64}:
  0.0
 -0.0
  0.07692307692307693

In [32]:
V = [2 0 0; 1 3 0; -2 2 1]

3×3 Matrix{Int64}:
  2  0  0
  1  3  0
 -2  2  1

In [36]:
K = V * transpose(V)

3×3 Matrix{Int64}:
  4   2  -4
  2  10   4
 -4   4   9

In [37]:
K * [1; 0; 1]

3-element Vector{Int64}:
 0
 6
 5